# ZSE Examples
## Jerry Crum 
### Updated 8/2/2020

First import all the modules we will use. Since zse is built around ase, import some ase modules as well. 

In [1]:
from ase.io import read,write
from ase.visualize import view
from zse import substitute, protonate, rings, cation
from zse.collections import framework

Get a zeolite framework to work with as an Atoms object.

In [2]:
atoms = framework('CHA')

Let's change one of the T-Sites from silicon to aluminum.

In [3]:
atoms = substitute.tsub(atoms, 101, 'Al')

Now that we have aluminum in the framework, let's add a charge compensating proton. That proton can bind to any of the four oxygens neighboring the aluminum, and this command will enumerate all the structures. The structures will be returned as a trajectory, and they will also be saved to the path provided. Each structure will be put into its own folder titled 'D-INDEX', where INDEX is the index of the oxygen that the proton is bound to.

In [4]:
traj, o = protonate.isolated(atoms,101,path='isolated')

I like to view the trajectory file to make sure everything looks ok. 

In [5]:
view(traj)

Now, we can use the ring counting tool to classify the oxygens that were used in the previous step. get_rings requires the atoms object, the index of the oxygen, and the types of rings possible for your framework (this information is included in the ZSE framework database or can be found on the IZA).

In [5]:
print(o)

[[11 19 50 62]]


In [5]:
possible_rings = rings.get_fwrings('CHA')
traj = []
for i in o[0]:
    c, paths, ring_atoms = rings.get_orings(atoms,i,possible_rings)
    print(i,c)
    traj+=[ring_atoms]

11 [6, 4, 4]
19 [8, 8, 4]
50 [8, 6, 4]
62 [8, 4, 4]


In [6]:
view(traj)

If you were curious, you can actually see the indices of the atoms that make up each of those rings. These paths are twice as long as the size of the ring because we only count T Site atoms when describing the rings. Blame the scientific community, not me. 

In [8]:
for i in o[0]:
    c, paths, ring_atoms = rings.get_orings(atoms,i,possible_rings)
    print('Oxygen =',i,c)
    for p in paths:
        print(p)

Oxygen = 11 [6, 4, 4]
[101, 50, 95, 14, 107, 53, 98, 17, 104, 47, 92, 11]
[101, 62, 80, 44, 89, 71, 92, 11]
[101, 19, 73, 1, 82, 28, 92, 11]
Oxygen = 19 [8, 8, 4]
[101, 62, 80, 26, 102, 45, 522, 461, 515, 497, 530, 463, 85, 37, 73, 19]
[101, 50, 95, 34, 304, 286, 307, 243, 297, 39, 75, 21, 106, 55, 73, 19]
[101, 11, 92, 28, 82, 1, 73, 19]
Oxygen = 50 [8, 6, 4]
[101, 19, 73, 55, 106, 21, 75, 39, 297, 243, 307, 286, 304, 34, 95, 50]
[101, 11, 92, 47, 104, 17, 98, 53, 107, 14, 95, 50]
[101, 62, 80, 8, 86, 68, 95, 50]
Oxygen = 62 [8, 4, 4]
[101, 19, 73, 37, 85, 463, 530, 497, 515, 461, 522, 45, 102, 26, 80, 62]
[101, 50, 95, 68, 86, 8, 80, 62]
[101, 11, 92, 71, 89, 44, 80, 62]


The following example repeats this process for a set of paired Al in the CHA framework. 

In [10]:
atoms = framework('CHA')
atoms = substitute.tsub(atoms, [101, 98], 'Al')
traj, o = protonate.paired(atoms,'Al',path='paired')

In [12]:
for i in o[0]:
    for j in o[1]:
        c = [rings.get_orings(atoms,i,possible_rings)[0],rings.get_orings(atoms,j,possible_rings)[0]]
        print('{0}-{1}: {2}'.format(i,j,c))

17-11: [[6, 4, 4], [6, 4, 4]]
17-19: [[6, 4, 4], [8, 8, 4]]
17-50: [[6, 4, 4], [8, 6, 4]]
17-62: [[6, 4, 4], [8, 4, 4]]
31-11: [[8, 8, 4], [6, 4, 4]]
31-19: [[8, 8, 4], [8, 8, 4]]
31-50: [[8, 8, 4], [8, 6, 4]]
31-62: [[8, 8, 4], [8, 4, 4]]
53-11: [[8, 6, 4], [6, 4, 4]]
53-19: [[8, 6, 4], [8, 8, 4]]
53-50: [[8, 6, 4], [8, 6, 4]]
53-62: [[8, 6, 4], [8, 4, 4]]
65-11: [[8, 4, 4], [6, 4, 4]]
65-19: [[8, 4, 4], [8, 8, 4]]
65-50: [[8, 4, 4], [8, 6, 4]]
65-62: [[8, 4, 4], [8, 4, 4]]


What if you want to find the rings associated with a particular T Site? I've got you covered.

In [13]:
atoms = framework('CHA')
c,paths,ringatoms = rings.get_trings(atoms,77,possible_rings)

In [14]:
print('Size of rings associated with T1 are: ',c)
print('The atom indices that make those rings are: ')
for p in paths:
    print(p)
view(ringatoms)

Size of rings associated with T1 are:  [8, 8, 6, 4, 4, 4]
The atom indices that make those rings are: 
[77, 59, 104, 25, 511, 475, 520, 682, 743, 716, 734, 680, 744, 483, 105, 23]
[77, 41, 83, 29, 90, 177, 627, 573, 634, 589, 640, 558, 612, 162, 105, 23]
[77, 41, 83, 2, 74, 38, 86, 8, 80, 44, 89, 5]
[77, 59, 104, 47, 92, 71, 89, 5]
[77, 59, 104, 17, 98, 65, 83, 41]
[77, 23, 105, 12, 93, 35, 89, 5]


Add copper atoms to a framework with two Al atoms in it. You can change 'Cu' to any divalent cation.

In [16]:
atoms = framework('CHA')
atoms = substitute.tsub(atoms,[101,98],'Al')
traj = cation.divalent(atoms, 'Cu', path='copper')

In [17]:
view(traj)

This also works for monovalent cations. This is currently only set up to consider an isolated Al.

In [19]:
atoms = framework('CHA')
atoms = substitute.tsub(atoms,101,'Al')
traj = cation.monovalent(atoms,'Na',path='sodium')

In [20]:
view(traj)

There is some helpful data stored in the ZSE framework database such as possible rings, tsites, and tsite multiplicity. I'll use the MFI framework as the example since CHA only has one unique T Site.

In [13]:
possible_rings = rings.get_fwrings('MFI')
print(possible_rings)

[10  6  5  4]


In [15]:
tsites,tmult = rings.get_tsites('MFI')
for t,m in zip(tsites,tmult):
    print('{0} \t {1}'.format(t,m))

T1 	 8
T2 	 8
T3 	 8
T4 	 8
T5 	 8
T6 	 8
T7 	 8
T8 	 8
T9 	 8
T10 	 8
T11 	 8
T12 	 8


You can also find all the unique rings in a framework using ZSE. This will return three dictionaries. The first contains all the ring paths using T Site notation. The second contains the ring paths using atom indices. The third contains a trajectory for each ring type, so you can visualize the different rings. I will use AEI for this example, because CHA is boring with only one T Site. 

In [2]:
possible_rings = rings.get_fwrings('AEI')
trings, fullrings, traj = rings.unique_rings('AEI')
for p in possible_rings:
    print('Unique {0}-MRs: {1}'.format(p,len(trings[p])))
    for r in trings[p]:
        print(r)

Unique 8-MRs: 2
['T1', 'T3', 'T2', 'T1', 'T1', 'T2', 'T3', 'T1']
['T2', 'T3', 'T3', 'T2', 'T2', 'T3', 'T3', 'T2']
Unique 6-MRs: 1
['T1', 'T3', 'T2', 'T2', 'T3', 'T1']
Unique 4-MRs: 4
['T1', 'T3', 'T3', 'T2']
['T1', 'T2', 'T2', 'T1']
['T1', 'T1', 'T1', 'T1']
['T2', 'T3', 'T2', 'T3']


Now to view the trajectory for one of the ring types use:

In [3]:
view(traj[8])

# IZA Frameworks

All of the CIFs available at http://www.iza-structure.org/databases/ are available through a database in this package. The following cell shows you how to call one of those structures. 

In [16]:
atoms = framework('MFI')
view(atoms)

This is everything that is included so far. Please provide any suggetions to tools you would like to see added. If you would like to contribute to this package, email me at jcrum@nd.edu.